In [0]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, DoubleType

stock_schema = StructType(
    [
        StructField("ticker", StringType()),
        StructField("timestamp", TimestampType()),
        StructField("open", DoubleType()),
        StructField("high", DoubleType()),
        StructField("low", DoubleType()),
        StructField("close", DoubleType()),
        StructField("volume", DoubleType()),
        StructField("trade_count", DoubleType()),
        StructField("vwap", DoubleType()),
    ]
)

In [0]:
# s3_path = dbutils.widgets.get("s3_path")
s3_path = "s3://amzn-s3-stock-prediction/stocks_data/ingestion_date=2025-11-21/stocks_2025-11-21.parquet"

data_frame = spark.read.schema(stock_schema).parquet(s3_path, index=False) 
distinct_tickers = data_frame.select("ticker").distinct()

In [0]:
data_frame.show(10)

In [0]:
path_table="stock_prediction.default"
table_name="stocks"

data_frame.createOrReplaceTempView("source_stocks_temp")
distinct_tickers.createOrReplaceTempView("distinct_tickers_temp")

In [0]:
%sql
MERGE INTO stock_prediction.default.companies AS target
USING distinct_tickers_temp AS source
ON target.ticker = source.ticker

WHEN NOT MATCHED THEN
  INSERT (ticker) VALUES (source.ticker)

In [0]:
%sql
MERGE INTO stock_prediction.default.stocks AS target
USING (
    SELECT
        source_stocks_temp.*,
        stock_prediction.default.companies.id AS company_id
    FROM source_stocks_temp JOIN stock_prediction.default.companies 
    ON source_stocks_temp.ticker = stock_prediction.default.companies.ticker
) AS source
ON target.timestamp = source.timestamp AND target.company_id = source.company_id

WHEN NOT MATCHED THEN
    INSERT (
        `company_id`,
        `timestamp`,
        `open_price`,
        `high_price`,
        `low_price`,
        `close_price`,
        `volume`,
        `trade_count`,
        `vwap`
    )
    VALUES (
        source.company_id,
        source.timestamp,
        source.open,
        source.high,
        source.low,
        source.close,
        source.volume,
        source.trade_count,
        source.vwap
    )